# Get the construction graph of project PTT-Archiv

Beware, this version is not working with python3.12!

In [2]:
# Common imports
import numpy as np
import math

# Library imports
import geovdata.kit as kit
import geovdata.sparql as sparql
from pyvis.network import Network

# Global variables
eta = kit.Eta()
pk_project = 11181465

# Connect to a SPARQL endpoint
sparql.connect_geovistory(pk_project)

>> SPARQL endpoint of Geovistory project 11181465 set.


What do we want?
- A network chart linking all the constructions
- Only the constructions should appear on the chart
- The constructions nodes should be bigger for those with more links.
- We would like the constructions texts to be "Construction Label [new line] (definition)"

Given the specifications, in the end, we need to have a table with 2 columns, in which each line represent a link between 2 constructions ([ontome:c441](https://ontome.net/class/441/namespace/3)) linked by a [ontome:Link](https://ontome.net/class/1380/namespace/3). The content of the columns should be formated string containing the construction's label and definition.

Let's make this chart!

## Get the data from the SPARQL endpoint

In [3]:
data = sparql.query("""
    select ?constr1Label ?constr1Definition ?constr2Label ?constr2Definition
    where {
        
        # Information about (left) construction: constr1
        ?constr1 a ontome:c441 . # Correct class
        ?constr1 rdfs:label ?constr1Label . # Label
        ?constr1 ontome:p1762 / ontome:p1864 ?constr1Definition . # Definition
               
        # Information about (right) construction: constr2
        ?constr2 a ontome:c441 . # Correct class
        ?constr2 rdfs:label ?constr2Label . # Label
        ?constr2 ontome:p1762 / ontome:p1864 ?constr2Definition . # Definition

        # Express how constructions are related
        ?constr1 ontome:p2320i / ontome:p2320 ?constr2 . # Through a link
               
        # But remove Link to itself
        filter(?constr1 != ?constr2)
    }
""")

kit.infos(data)

Shape:  (138, 4) - extract:


,constr1Label,constr1Definition,constr2Label,constr2Definition
0,Knotenamt Gümmenen,telephone exchange,Fernknotenamt Mattenhof,Phonehouse structure
1,Knotenamt Gümmenen,telephone exchange,Endamt Kerzers,telephone exchange
2,Unterzentrale Bümplitz,telephone exchange,Fernknotenamt Mattenhof,Phonehouse structure
3,Unterzentrale Bümplitz,telephone exchange,Fernknotenamt Bollwerk,telephone exchange
4,Unterzentrale Bümplitz,telephone exchange,Quartierzentrale Breitenrain,telephone exchange


## Format strings

Now that we have all needed information, let's create our formated string.

It should be: "Label [newline] (definition)"

In [4]:
# Format label for construction 1
data['constr1'] = data['constr1Label'] + '\n(' + data['constr1Definition'] + ')'

# Format label for construction 2
data['constr2'] = data['constr2Label'] + '(' + data['constr2Definition'] + ')'

# Remove old columns
data = data[['constr1', 'constr2']]

kit.infos(data)

Shape:  (138, 2) - extract:


,constr1,constr2
0,Knotenamt Gümmenen\n(telephone exchange),Fernknotenamt Mattenhof(Phonehouse structure)
1,Knotenamt Gümmenen\n(telephone exchange),Endamt Kerzers(telephone exchange)
2,Unterzentrale Bümplitz\n(telephone exchange),Fernknotenamt Mattenhof(Phonehouse structure)
3,Unterzentrale Bümplitz\n(telephone exchange),Fernknotenamt Bollwerk(telephone exchange)
4,Unterzentrale Bümplitz\n(telephone exchange),Quartierzentrale Breitenrain(telephone exchange)


## Display the chart

In [5]:
# Create the network
network = Network(
    height=900, # In pixels
    width=1800, # In pixels
    notebook=True, 
    cdn_resources="in_line", 
    select_menu=True
)

# All the nodes
nodes = np.unique(data['constr1'].tolist() + data['constr2'].tolist())

# Node sizes
sizes = []
for node in nodes:
    nb_constr1 = (data['constr1'] == node).sum()
    nb_constr2 = (data['constr2'] == node).sum()
    sizes.append(10 * math.sqrt((nb_constr1 + nb_constr2) / (2 * math.pi)))

# Colors 
colors = ["#9c0ef3"] * len(nodes)

# Add the nodes
network.add_nodes(nodes, size=sizes, color=colors)

# All the edges
edges = []
for _, row in data.iterrows():
    edges.append((row['constr1'], row['constr2']))

# Add the edges
network.add_edges(edges)

# Set Chart options
network.toggle_physics(True)
network.show_buttons(filter_=['physics'])

# Save the file (automatic open)
path = network.show('./construction-graph.html', local=True, notebook=False)

./construction-graph.html


You can now open the just created html file to see your graph.